In [1]:
import pandas as pd
df = pd.read_csv('../data/Original_csv/Hyperion.csv')


In [2]:
import string

def find_char_bounds(spans: list, text: str) -> list:
    '''
    Given a list of spans and a text, find the start and end indices of each span in the text.
    Indeces are computed counting chars
    
    :param spans: a list of strings to search for
    :type spans: list
    :param text: the text to search
    :type text: str
    :return: A list of tuples, where each tuple contains the start and end index of a span.
    '''
    bounds = []
    last_char = 0
    for span in spans:
        start = text.find(span)
        if start == -1:
            start = last_char + 1
        bounds.append((start, start + len(span)-1))
        last_char = start + len(span)-1
    return bounds


def IE_gen(bounds: list, text:str) -> str:
    tags = ['I' for i in range(len(text))]
    for bound in bounds:
        if bound[1] < len(text):
            tags[bound[1]] = 'E'
        else:
            tags[-1] = 'E'
    return ''.join(tags)


In [3]:
from transformers import AutoTokenizer
import re

dataset = []

for row in df.itertuples():
    text = row.Testo
    
    if pd.isna(text):
        span =  re.sub(r'[\?\.\!]+(?=[\?\.\!])', '', row.Stralcio)
        sample['Stralci'].append(span)
        sample['Repertori'].append(row.Repertorio)

    else:
        sample = {}
        text = re.sub(r'[\?\.\!]+(?=[\?\.\!])', '', text)
        sample['Testo'] = text
        span =  re.sub(r'[\?\.\!]+(?=[\?\.\!])', '', row.Stralcio)
        sample['Stralci'] = [span]

        sample['Repertori'] = [row.Repertorio]
        dataset.append(sample)



for i,sample in enumerate(dataset):
    sample['Bounds'] = find_char_bounds(sample['Stralci'], sample['Testo'])
    sample['Tags'] = IE_gen(sample['Bounds'], sample['Testo'])


In [4]:
IE_dict = {
    'Testo': [sample['Testo'] for sample in dataset],
    'Tags': [sample['Tags'] for sample in dataset],
    'Bounds': [sample['Bounds'] for sample in dataset],
    'Repertori': [sample['Repertori'] for sample in dataset],
    'Stralci': [sample['Stralci'] for sample in dataset]
}
IE_df = pd.DataFrame(IE_dict)
IE_df = IE_df.head(1000)

In [9]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn import preprocessing
import numpy as np
import torch

LABELS = [
    'I',
    'E'
]


def encode_labels(labels_list):
    le = preprocessing.LabelEncoder()
    le.fit(LABELS)
    return le.transform(labels_list)


class IE_Hyperion_dataset(Dataset):
    def __init__(self, df, tokenizer_name):
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.df = df

    def __getitem__(self, idx):
        text = self.df['Testo'].iloc[idx]
        encoding = self.tokenizer(text,
                                  # is_pretokenized=True,
                                  return_special_tokens_mask=True,
                                  return_offsets_mapping=True,
                                  add_special_tokens=True,
                                  return_attention_mask=True,
                                  padding='max_length',
                                  truncation=True,
                                  )
        char_labels = encode_labels(list(self.df['Tags'].iloc[idx]))
        ends = [i for i in range(len(char_labels)) if char_labels[i] == 0]

        last_token_idx = max(index for index, item in enumerate(encoding['special_tokens_mask']) if item == 0)

        encoded_labels = np.ones(len(encoding['input_ids']), dtype=int) * -100
        x = ends.pop(0)
        for i,e in enumerate(encoding['offset_mapping']):
            if e[1] != 0:
                # overwrite label
                if x >= e[0] and x <= e[1]:
                    encoded_labels[i] = 0
                    if ends: 
                        x = ends.pop(0)
                    else:
                        x = -1
                else:
                    encoded_labels[i] = 1
        if not 0 in encoded_labels:
            encoded_labels[last_token_idx] = 0

        

        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        return item

    def __len__(self):
        return len(self.df.index)


In [10]:
model_name = "dbmdz/bert-base-italian-xxl-uncased"
train_size = 0.8
train_df = IE_df.sample(frac=train_size, random_state=200)
test_df = IE_df.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)
val_size = 0.2
val_df = train_df.sample(frac=val_size, random_state=200)
train_df = train_df.drop(val_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

print("FULL Dataset: {}".format(IE_df.shape))
print("TRAIN Dataset: {}".format(train_df.shape))
print("VALIDATION Dataset: {}".format(val_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

train_dataset = IE_Hyperion_dataset(train_df, model_name)
val_dataset = IE_Hyperion_dataset(val_df, model_name)
test_dataset = IE_Hyperion_dataset(test_df, model_name)


FULL Dataset: (1000, 5)
TRAIN Dataset: (640, 5)
VALIDATION Dataset: (160, 5)
TEST Dataset: (200, 5)


In [11]:
print(test_dataset.df['Bounds'].iloc[20])
print(test_dataset.df['Repertori'].iloc[20])
print(test_dataset.df['Testo'].iloc[20])
print(test_dataset.df['Stralci'].iloc[20])

[(0, 270), (271, 357), (360, 389), (390, 450), (439, 511), (512, 587), (577, 587)]
['sancire', 'giudizio', 'commento', 'sancire', 'commento', 'giudizio', 'commento']
Non passerà un bel niente ! La strage di morti non si può dimenticare come non si può dimenticare chi a gennaio diceva che non c’è pericolo che è poco più di un influenza! Con un influenza non muoiono in un mese e mezzo 23000 persone , non si svuotano le case di riposo !ci sono dei responsabili per tutto questo ! Responsabili di omicidio colposo epidemico ! E non mi rallegra le nascite ! Non rimpiazzano chi è morto solo come un cane ! Se io fossi giovane non metterei al mondo dei figli per offrirgli cosa ? Un mondo senza futuro di ingiustizie , intollerante e violenze ! No grazie !
['Non passerà un bel niente ! La strage di morti non si può dimenticare come non si può dimenticare chi a gennaio diceva che non c’è pericolo che è poco più di un influenza! Con un influenza non muoiono in un mese e mezzo 23000 persone , non si 

In [13]:
for token, label in zip(test_dataset.tokenizer.convert_ids_to_tokens(test_dataset[197]["input_ids"]), test_dataset[197]["labels"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       -100
la          1
mappa       1
del         1
potere      1
durante     1
l           1
’           1
emergenza   1
co          1
##vid       1
-           1
19          1
in          1
veneto      1
i           1
politici    1
,           1
i           1
dirigenti   1
regionali   1
e           1
gli         1
esperti     1
che         1
compongono  1
l           1
'           1
unita       1
di          1
crisi       1
e           1
gli         1
altri       1
organi      1
creati      1
per         1
gestire     1
l           1
'           1
emergenza   1
sanitaria   1
.           1
nella       1
mappa       1
del         1
potere      1
che         1
ha          1
gestito     1
l           1
’           1
emergenza   1
corona      1
##virus     1
il          1
ruolo       1
centrale    1
e           1
attribuito  1
al          1
presidente  1
za          1
##ia        1
in          1
qualita     1
di          1
soggetto    1
attuato     1
##re        1
.           1
a  

In [14]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=2)


Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not in

In [15]:
import random
import os

import torch
import numpy as np
import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


# Deterministic mode
def seed_everything(seed=1464):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


def plot_loss(loss, val_loss):
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.xticks(range(1, len(loss)+1))
    plt.plot(range(1, len(loss)+1), loss, label='train')
    plt.plot(range(1, len(val_loss)+1), val_loss, label='val')
    plt.title('loss')
    plt.legend()
    # plt.savefig('loss.png')
    return fig


def plot_f1(f1, val_f1):
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.xticks(range(1, len(f1)+1))
    plt.plot(range(1, len(f1)+1), f1, label='train')
    plt.plot(range(1, len(val_f1)+1), val_f1, label='val')
    plt.title('f1')
    plt.legend()
    # plt.savefig('f1.png')
    return fig


def plot_confusion_matrix(y_true, pred, labels):
    fig, ax = plt.subplots(figsize=(20, 20))
    disp = ConfusionMatrixDisplay.from_predictions(
        y_true, pred, display_labels=labels, normalize='true', values_format='.2f')
    disp.plot(cmap="Blues", values_format='.2g',
              xticks_rotation='vertical', ax=ax)
    return disp.figure_


In [16]:
import neptune.new as neptune


class NeptuneLogger():
    def __init__(self) -> None:
        # Neptune initialization
        self.run = neptune.init(
            project="mibo8/Rep",
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmZmRkYThiZi1mZGNlLTRlMTktODQwNS1hNWFlMWQ2Mjc4N2IifQ==",
        )


In [17]:
import time
import datetime
from torch.nn import utils

import torchmetrics
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import get_constant_schedule_with_warmup


import neptune.new as neptune


class IE_MPTrainer():
    def __init__(self, batch_size, lr, n_epochs) -> None:
        self.batch_size = batch_size
        self.learning_rate = lr
        self.n_epochs = n_epochs

        self.logger = NeptuneLogger()

    def fit(self, model, train_dataset, val_dataset):
        self.logger.run['model'] = model_name

        params_info = {
            'learning_rate': self.learning_rate,
            'batch_size': self.batch_size,
            'n_epochs': self.n_epochs
        }
        #self.logger.run['params'] = params_info

        torch.cuda.empty_cache()
        # ----------TRAINING

        # Measure the total training time for the whole run.
        total_t0 = time.time()

        epochs_train_loss = []
        epochs_val_loss = []

        epochs = self.n_epochs

        # Creation of Pytorch DataLoaders with shuffle=True for the traing phase
        train_dataloader = DataLoader(
            train_dataset, batch_size=self.batch_size, shuffle=True)
        validation_dataloader = DataLoader(
            val_dataset, batch_size=self.batch_size, shuffle=True)

        # Adam algorithm optimized for tranfor architectures
        optimizer = AdamW(model.parameters(), lr=self.learning_rate)
        #scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=300)

        # Scaler for mixed precision
        scaler = torch.cuda.amp.GradScaler()

        # Setup for training with gpu
        device = torch.device(
            'cuda') if torch.cuda.is_available() else torch.device('cpu')
        model.to(device)

        # For each epoch...
        for epoch_i in range(0, epochs):

            # ========================================
            #               Training
            # ========================================

            # Perform one full pass over the training set.

            print("")
            print(
                '======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            print('Training...')

            # Measure how long the training epoch takes.
            t0 = time.time()

            # Reset the total loss for this epoch.
            total_train_loss = 0

            # Put the model into training mode: Dropout layers are active
            model.train()

            # For each batch of training data...
            for step, batch in enumerate(train_dataloader):

                # Progress update every 40 batches.
                if step % 10 == 0 and not step == 0:
                    # Compute time in minutes.
                    elapsed = format_time(time.time() - t0)

                    # Report progress.
                    print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(
                        step, len(train_dataloader), elapsed))

                # Unpack this training batch from the dataloader.
                #
                #  copy each tensor to the GPU using the 'to()' method
                #
                # 'batch' contains three pytorch tensors:
                #   [0]: input ids
                #   [1]: attention masks
                #   [2]: labels
                b_input_ids = batch['input_ids'].to(device)
                b_input_mask = batch['attention_mask'].to(device)
                b_labels = batch['labels'].to(device)

                # clear any previously calculated gradients before performing a
                # backward pass
                model.zero_grad()

                # Perform a forward pass in mixed precision
                with torch.cuda.amp.autocast():
                    outputs = model(b_input_ids,
                                    attention_mask=b_input_mask,
                                    labels=b_labels)

                    loss = outputs[0]
                    logits = outputs[1]

                # Move logits and labels to CPU
                logits = logits.detach().cpu()
                label_ids = b_labels.to('cpu')

                # Perform a backward pass to compute the gradients in MIXED precision
                scaler.scale(loss).backward()

                # Accumulate the training loss over all of the batches so that we can
                # calculate the average loss at the end.
                total_train_loss += loss.item()

                # Unscales the gradients of optimizer's assigned params in-place before the gradient clipping
                scaler.unscale_(optimizer)

                # Clip the norm of the gradients to 1.0.
                # This helps and prevent the "exploding gradients" problem.
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                # Update parameters and take a step using the computed gradient in MIXED precision
                scaler.step(optimizer)
                scaler.update()
                # scheduler.step()

            # Compute the average loss over all of the batches.
            avg_train_loss = total_train_loss / len(train_dataloader)
            epochs_train_loss.append(avg_train_loss)

            # Measure how long this epoch took.
            training_time = format_time(time.time() - t0)

            print("")
            print("  Average training loss: {0:.3f}".format(avg_train_loss))
            print("  Training epoch took: {:}".format(training_time))

            # ========================================
            #               Validation
            # ========================================
            # After the completion of each training epoch, measure performance on
            # the validation set.

            print("")
            print("Running Validation...")

            t0 = time.time()

            # Put the model in evaluation mode: the dropout layers behave differently
            model.eval()

            total_val_loss = 0

            # Evaluate data for one epoch
            for batch in validation_dataloader:

                # Unpack this training batch from our dataloader.
                #
                # copy each tensor to the GPU using the 'to()' method
                #
                # 'batch' contains three pytorch tensors:
                #   [0]: input ids
                #   [1]: attention masks
                #   [2]: labels
                b_input_ids = batch['input_ids'].to(device)
                b_input_mask = batch['attention_mask'].to(device)
                b_labels = batch['labels'].to(device)

                # Tell pytorch not to bother with constructing the compute graph during
                # the forward pass, since this is only needed for training.
                with torch.no_grad():

                    # Forward pass, calculate logits
                    # argmax(logits) = argmax(Softmax(logits))
                    outputs = model(b_input_ids,
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
                    loss = outputs[0]
                    logits = outputs[1]

                # Accumulate the validation loss.
                total_val_loss += loss.item()

                # Move logits and labels to CPU
                logits = logits.detach().cpu()
                label_ids = b_labels.to('cpu')

            print('VALIDATION: ')

            # Compute the average loss over all of the batches.
            avg_val_loss = total_val_loss / len(validation_dataloader)
            epochs_val_loss.append(avg_val_loss)

            # Measure how long the validation run took.
            validation_time = format_time(time.time() - t0)

            print("  Validation Loss: {0:.2f}".format(avg_val_loss))
            print("  Validation took: {:}".format(validation_time))

        loss_fig = plot_loss(epochs_train_loss, epochs_val_loss)

        self.logger.run["loss"].upload(neptune.types.File.as_image(loss_fig))
        print("")
        print("Training complete!")

        print("Total training took {:} (h:mm:ss)".format(
            format_time(time.time()-total_t0)))

    def test(self, model, test_dataset):
        # ========================================
        #               Test
        # ========================================
        test_dataloader = DataLoader(
            test_dataset, batch_size=self.batch_size, shuffle=False)

        # Setup for testing with gpu
        device = torch.device(
            'cuda') if torch.cuda.is_available() else torch.device('cpu')

        print("")
        print("Running Test...")
        t0 = time.time()

        # Save prediction for confusion matrix
        pred = []

        model.eval()

        total_test_loss = 0

        # Evaluate data for one epoch
        for batch in test_dataloader:
            b_input_ids = batch['input_ids'].to(device)
            b_input_mask = batch['attention_mask'].to(device)
            b_labels = batch['labels'].to(device)
            b_special_tokens_mask = batch['special_tokens_mask'].to(device)
            with torch.no_grad():

                # Forward pass, calculate logits
                # argmax(logits) = argmax(Softmax(logits))
                outputs = model(b_input_ids,
                                attention_mask=b_input_mask,
                                labels=b_labels)
                loss = outputs[0]
                logits = outputs[1]

            # Accumulate the test loss.
            total_test_loss += loss.item()

            # Move logits and labels to CPU
            logits = logits.detach().cpu()  # shape (batch_size, seq_len, num_labels)
            label_ids = b_labels.to('cpu')

            batch_pred = logits.softmax(dim=-1)
            full_pred = batch_pred.argmax(dim=-1)

            for i, sample_pred in enumerate(full_pred):
                active_pred = []
                for j, e in enumerate(b_special_tokens_mask[i]):
                    if(e == 0):
                        active_pred.append(int(sample_pred[j]))
                pred.append(active_pred)

        avg_test_loss = total_test_loss / len(test_dataloader)
        #self.logger.run['test/loss'] = avg_test_loss
        test_time = format_time(time.time() - t0)

        print("  Test Loss: {0:.2f}".format(avg_test_loss))
        print("  Test took: {:}".format(test_time))

        return pred


In [18]:
# Hyperparameters
learning_rate = 1e-5
batch_size = 2
n_epochs = 1
trainer = IE_MPTrainer(batch_size, learning_rate, n_epochs)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs.neptune.ai/you-should-know/what-can-you-log-and-display#hardware-consumption


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [19]:
#trainer.fit(model, train_dataset, val_dataset)


In [20]:
pred = trainer.test(model, test_dataset)


Running Test...
  Test Loss: 0.68
  Test took: 0:02:29


In [21]:
def prediction_to_bounds(pred:list) -> list:
    bounds = []
    start = 0
    end = 0
    for i,e in enumerate(pred):
        if e == 0:
            end = i
            bounds.append((start, end))
            start = end + 1
    return bounds


bert_preds = []
for e in pred:
    bert_preds.append(prediction_to_bounds(e))

gt_bounds = []
for e in test_dataset:
    gt_bounds.append(prediction_to_bounds(e['labels']))
test_dataset.df['Token_bounds'] = gt_bounds

In [23]:
counter = 0
for i in range(len(test_dataset.df.index)):
    if len(test_dataset.df.iloc[i]['Token_bounds']) != len(test_dataset.df.iloc[i]['Repertori']):
        counter += 1
print(counter)

13


In [24]:
print(test_dataset.df.iloc[0]['Repertori'])
print(test_dataset.df.iloc[0]['Token_bounds'])

['dichiarazione di intenti', 'sancire', 'giustificazione', 'sancire', 'commento', 'giudizio']
[(0, 5), (6, 16), (17, 29), (30, 48), (49, 57), (58, 60)]


In [25]:
import numpy as np
# A è B sono tupe con i bound dello span
def IoU(A, B):
    '''
    Given two intervals, find the intersection over union between them.
    
    :param A: The first bounding box
    :param B: The bounding box
    :return: the intersection over union of the two bounding boxes.
    '''
    if A == B:
        return 1
    start = max(A[0], B[0])
    end = min(A[1], B[1])
    if(start > end):
        return 0
    intersection = end - start
    return intersection / (A[1] - A[0] + B[1] - B[0] - intersection)

def compute_IoUs(pred_bounds, gt_spans):
    '''
    Given a list of predicted spans and a list of ground truth spans, 
    compute the intersection over union for each pair of spans
    
    :param pred_bounds: a tuple of (start, end) denoting the predicted answer
    :param gt_spans: a list of tuples of the form (start, end) representing the spans of each ground
    truth annotation
    :return: a list of IoUs for each ground truth span.
    '''
    IoUs = []
    for gt_bounds in gt_spans:
        IoUs.append(IoU(pred_bounds, gt_bounds)) 
    return IoUs

def normalize(text_spans_dict, gt_spans):
    normalized = []
    for i in range(len(text_spans_dict)):
        #normalized is not empty
        if normalized:
            if normalized[-1]['Repertorio'] == text_spans_dict[i]['Repertorio']:
                new_span = (normalized[-1]['Bounds'][0], text_spans_dict[i]['Bounds'][1])
                new_span_features = {
                    'Bounds' : new_span, 
                    'IoU' : None,
                    'Repertorio' : text_spans_dict[i]['Repertorio']
                    }
                del normalized[-1]
                normalized.append(new_span_features)
            else:
                normalized.append(text_spans_dict[i])
        else:
            normalized.append(text_spans_dict[i])
        
    
    for i in range(len(normalized)):
        normalized[i]['IoU'] = max(compute_IoUs(normalized[i]['Bounds'], gt_spans['Token_bounds']))
    return normalized
    

metrics = []
normalized_metrics = []
for i, pred_bounds in enumerate(bert_preds):
    text_IoUs = []
    for pred_span in pred_bounds:
        IoUs = compute_IoUs(pred_span, test_dataset.df.iloc[i]['Token_bounds'])   
        best = np.argmax(IoUs)
        span_features = {
            'Bounds' : pred_span, 
            'IoU' : IoUs[best],
            'Repertorio' : test_dataset.df.iloc[i]['Repertori'][best]
            }

        text_IoUs.append(span_features)
    metrics.append(text_IoUs)
    normalized_metrics.append(normalize(text_IoUs, test_dataset.df.iloc[i]))

In [28]:
print('----------------------------------------------------------')
print('Risultati labels GT e stralci non uniti')


n_spans = 0
for e in test_dataset.df['Token_bounds']:
    n_spans += len(e)
print('Numero stralci nel dataset:', str(n_spans))

n_spans = 0
for e in metrics:
    n_spans += len(e)
print('Numero stralci predetti:', str(n_spans))

mean = 0
long_spans = 0
min_lenght = 0
perfect_spans = 0
for text in metrics:
    for span in text:
        if span['Bounds'][1] - span['Bounds'][0] >= min_lenght:
            long_spans += 1
            mean += span['IoU']
            if span['IoU'] == 1:
                perfect_spans += 1
perfect_spans_perc = perfect_spans / long_spans
mean_IoU = mean / long_spans
print('Numero stralci con lunghezza minima = ',
      str(min_lenght), ': ', str(long_spans))
print('Media IoU:', str(mean_IoU))
print('Percentuale span perfetti: ', str(perfect_spans_perc))


----------------------------------------------------------
Risultati labels GT e stralci non uniti
Numero stralci nel dataset: 647
Numero stralci predetti: 8806
Numero stralci con lunghezza minima =  0 :  8806
Media IoU: 0.039310584196279676
Percentuale span perfetti:  0.00034067681126504656


In [29]:
print('----------------------------------------------------------')
print('Risultati labels GT e stralci uniti')


n_spans = 0
for e in test_df['Token_bounds']:
    n_spans += len(e)
print('Numero stralci nel dataset:', str(n_spans))

n_spans = 0
for e in normalized_metrics:
    n_spans += len(e)
print('Numero stralci predetti:', str(n_spans))

mean = 0
long_spans = 0
min_lenght = 0
perfect_spans = 0
for text in normalized_metrics:
    for span in text:
        if span['Bounds'][1] - span['Bounds'][0] >= min_lenght:
            long_spans += 1
            mean += span['IoU']
            if span['IoU'] == 1:
                perfect_spans += 1
perfect_spans_perc = perfect_spans / long_spans
mean_IoU = mean / long_spans
print('Numero stralci con lunghezza minima = ',
      str(min_lenght), ': ', str(long_spans))
print('Media IoU:', str(mean_IoU))
print('Percentuale span perfetti: ', str(perfect_spans_perc))


----------------------------------------------------------
Risultati labels GT e stralci uniti
Numero stralci nel dataset: 647
Numero stralci predetti: 2116
Numero stralci con lunghezza minima =  0 :  2116
Media IoU: 0.23227367464258894
Percentuale span perfetti:  0.02835538752362949
